In [1]:
%run ../Requirements.ipynb

24/06/20 15:47:48 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
24/06/20 15:47:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 15:47:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/20 15:47:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
auth_header = base64.b64encode((spotify_secrets.get("client_id")+':'+spotify_secrets.get("client_secret")).encode()).decode()
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_header}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(spotify_auth_url, headers=headers, data=data)
response_data = response.json()['access_token']

headers = {
    "Authorization": f"Bearer {response_data}"
}

response = requests.get(spotify_url, headers=headers)
spotify_response_data=response.json()

In [3]:
tracks = []
for item in spotify_response_data['tracks']['items']:
    track = item['track']
    track_info = {
        'Track Name': track['name'],
        'Artists': ', '.join([artist['name'] for artist in track['artists']]),
        'Album': track['album']['name'],
        'Release Date': track['album']['release_date'],
        'Track URL': track['external_urls']['spotify']
    }
    tracks.append(track_info)
df = pd.DataFrame(tracks)
df = spark.createDataFrame(tracks)

In [4]:
df.write.mode('overwrite').format('parquet').save('../Staging/SpotifyMusic')

24/06/20 15:47:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
